In [32]:
from random import randint, choice
from pprint import pprint
from itertools import permutations
from math import inf as oo # Infinity (∞) is larger than any number
from math import sqrt
from time      import time
import matplotlib.pyplot as plt
import copy

MAX_DISTANCE = 100

def random_symmetric_graph(n):
    ''' Symmetric adjacency matrix of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            v = randint(1,MAX_DISTANCE)
            dist_matrix[i][j] = v
            dist_matrix[j][i] = v
    return dist_matrix

def random_euclidean_graph(n):
    ''' Symmetric adjacency matrix of a Euclidean graph of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    points = []
    for p in range(n):
        x,y = randint(0,MAX_DISTANCE), randint(0,MAX_DISTANCE)
        points.append((x,y))
    for i in range(n):
        p1 = points[i]
        for j in range(i+1,n):
            p2 = points[j]
            distance = sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            dist_matrix[i][j] = distance
            dist_matrix[j][i] = distance
    return dist_matrix

def show(G):
    ''' Show adjacency matrix. Useful for debugging. '''
    n = len(G)
    r = "     "
    for i in range(n):
        r += f'{i:4}'
    r += '\n    -'+'-'*(4*n)+'\n'
    for i in range(n):
        r += f'{i:2} | '
        for j in range(n):
            r += f'{G[i][j]:4}'
        r += '\n'
    r = r.replace('inf', '  ∞')
    print(r)

def cost(G, cycle):
    ''' Calculate the cost of the given cycle '''
    c = 0
    n = len(G)
    for i in range(n):
        a = cycle[i]
        b = cycle[(i+1)%n]
        c += G[a][b]
    return c

In [112]:
G = random_symmetric_graph(8)

In [113]:
show(G)

        0   1   2   3   4   5   6   7
    ---------------------------------
 0 |    ∞  96  67  39  80  99  93  56
 1 |   96   ∞  37  30  41  34  21  28
 2 |   67  37   ∞  28  86  88  66   4
 3 |   39  30  28   ∞  65  46  56  93
 4 |   80  41  86  65   ∞  16  63  75
 5 |   99  34  88  46  16   ∞  75   3
 6 |   93  21  66  56  63  75   ∞  49
 7 |   56  28   4  93  75   3  49   ∞



In [150]:
cycle = [0,1,2,3,4,5,6,7]

In [151]:
cost(G, cycle)

422

In [134]:
def exhaustive_search(G):
    n = len(G)    
    best_cost = oo # infinity
    best_cycle = []
    for cycle in permutations(range(1,n)): # permutations of [1,2,...,n-1]
        cycle=[0]+list(cycle)              # add the starting city: 0
        c = cost(G, cycle)
        if c < best_cost:
            best_cost = c
            best_cycle = cycle
    return (best_cycle, best_cost)

In [194]:
g = G[0]
g
# g = min(g)
# G.index(g)

[inf, 96, 67, 39, 80, 99, 93, 56]

# Greedy Randomized Adaptive Search Procedure (GRASP)

In [156]:
def GreedySolution(G):
    H = copy.deepcopy(G) # We need the original G. We work on/modify H
    n = len(H)
    cities = list(range(n))
    cycle = [] # solution to be built
    city = 0 # Start city
    while len(cities)>0:
        # Find nearest neighbour
        city_neighbours = H[city]
        smallest_distance = min(city_neighbours)
        nearest_city = city_neighbours.index(smallest_distance)
        # Update 'cycle' and 'cities' and H then 'city'
        cycle.append(city)
        cities.remove(city)
        for i in range(n): # 'city' is not to be used again!
            H[city][i] = oo
            H[i][city] = oo
        city = nearest_city
    return (cycle)


In [ ]:
def LocalSearch(nodes, Solution, maxcount):
    count = 0 
    newsolution = []
    while count <= maxcount:
        newsolution = Solution
        node_neighbours = nodes[count]
        smallest_distance = min(node_neighbours)
        nearest_city = node_neighbours.index(smallest_distance)
        newsolution[count] = nodes[nearest_city]
        if cost(nodes, Solution) < 
#         help
        
        

In [ ]:
def grasp(Max_Iterations, G):
    CurrentBest = []
    for count in range(Max_Iterations):
        Solution = GreedySolution(G)
        NewSolution = LocalSearch(Solution)
        if NewSolution < CurrentBest:
            CurrentBest = NewSolution
    return CurrentBest


    
    
    


In [177]:
LocalSearch(G, GreedySolution(G), 10)

UnboundLocalError: local variable 'nearest' referenced before assignment